In [282]:
# Biblioteki wejściowe

import pandas as pd
import numpy as np
import numpy_financial as npf
import posixpath as os
import pathlib
from pathlib import Path
import datetime
import sys
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd
import IPython
import math

from datetime import date
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression

pd.set_option("display.max_rows", None)
#pd.options.display.max_rows=50000
pd.options.display.max_colwidth=70
#pd.options.display.min_colwidth=10
pd.options.display.precision=2
pd.options.display.max_columns=50
pd.options.display.float_format='{0:,.2f}'.format

DATA = Path(r'/Users/daniel_wojcik/Documents/QFA/TEZET_Terminal/DATA')
ASSETS = Path(r'/Users/daniel_wojcik/Documents/QFA/TEZET_Terminal/ASSETS')
IMAGES = Path(r'/Users/daniel_wojcik/Documents/QFA/TEZET_Terminal/IMAGES')

In [283]:
# Dane wejściowe modelu

# zakres dat
DATE_START=pd.to_datetime('2023-01-31')
DATE_R=DATE_START.year
TENOR_YEARS=5
DATE_END=datetime.datetime(DATE_R+TENOR_YEARS,12,31)

# parmetry finansowe
FX_EUR=4.82
DISCOUNT_RATE=9.0

INDEX=7.12
MARGIN=2.5

# zakres czasowy modelu
okres=pd.date_range(start=DATE_START,freq='M',end=DATE_END)
t_okres=pd.DataFrame({'DATE':okres})
t_okres['R']=t_okres['DATE'].dt.year
t_okres['M']=t_okres['DATE'].dt.month
t_okres['I']=t_okres.index

t_model=t_okres

#### Istniejące aktywa i pasywa

In [284]:
# Istniejące aktywa trwałe

l_at=[
        {'Nazwa':                   'Wartości niematerialne i prawne',
        'WNA [PLN]':                3500000,
        'Amortyzacja [PLN]':        3000000},

        {'Nazwa':                   'Grunty',
        'WNA [PLN]':                7750000,
        'Amortyzacja [PLN]':        0},

        {'Nazwa':                   'Budynki i budowle',
        'WNA [PLN]':                13100000,
        'Amortyzacja [PLN]':        1310000},

        {'Nazwa':                   'Urządzenia techniczne i maszyny',
        'WNA [PLN]':                2600000,
        'Amortyzacja [PLN]':        364000},

        {'Nazwa':                   'Środki transportu',
        'WNA [PLN]':                320000,
        'Amortyzacja [PLN]':        64000},

        {'Nazwa':                   'Inne środki trwałe',
        'WNA [PLN]':                0,
        'Amortyzacja [PLN]':        0},

        {'Nazwa':                   'Środki trwałe w budowie',
        'WNA [PLN]':                1600000,
        'Amortyzacja [PLN]':        0}

]

t_at=pd.DataFrame(l_at)

t_tabela=t_at.copy()
t_tabela=t_tabela[['Nazwa', 'WNA [PLN]', 'Amortyzacja [PLN]']]

t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Nazwa']='RAZEM'  
t_tabela['WNA [PLN]']=t_tabela['WNA [PLN]']/1000
t_tabela['Amortyzacja [PLN]']=t_tabela['Amortyzacja [PLN]']/1000

t_tabela_fv=t_tabela.style.format({'WNA [PLN]':'{0:,.2f}','Amortyzacja [PLN]':'{0:,.2f}'}).hide(axis='index')
t_tabela_fv.to_latex(ASSETS / '1_Istniejące_AT.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')
t_tabela_fv

Nazwa,WNA [PLN],Amortyzacja [PLN]
Wartości niematerialne i prawne,"3,500.00","3,000.00"
Grunty,"7,750.00",0.00
Budynki i budowle,"13,100.00","1,310.00"
Urządzenia techniczne i maszyny,"2,600.00",364.00
Środki transportu,320.00,64.00
Inne środki trwałe,0.00,0.00
Środki trwałe w budowie,"1,600.00",0.00
RAZEM,"28,870.00","4,738.00"


In [285]:
# Rozliczenie AT

DATA_BO=pd.to_datetime('2023-01-31')

t_at_rozliczenie=pd.DataFrame()

def AmPeriod(wna,am):
    if am!=0:
        return int((wna/am)*12)
    else:
        return 0

for index,row in t_at.iterrows():
    NAME=row['Nazwa']
    WNA=row['WNA [PLN]']
    AM=row['Amortyzacja [PLN]']

    if AM!=0:
        okres=pd.date_range(start=DATA_BO,freq='M',periods=AmPeriod(WNA,AM))
        t_at_temp=pd.DataFrame({'DATE':okres})
        t_at_temp['R']=t_at_temp['DATE'].dt.year
        t_at_temp['M']=t_at_temp['DATE'].dt.month
        t_at_temp['I']=t_at_temp.index
        t_at_temp['Nazwa']=NAME
        t_at_temp['Aktywa_Trwałe_BO']=t_at_temp.apply(lambda x:0 if x['DATE']!=DATA_BO else WNA, axis=1)
        t_at_temp['Amortyzacja_BO']=-AM/12
        t_at_rozliczenie=pd.concat([t_at_rozliczenie,t_at_temp],ignore_index=True)
    else:
        okres=pd.date_range(start=DATA_BO,freq='M',periods=1)
        t_at_temp=pd.DataFrame({'DATE':okres})
        t_at_temp['R']=t_at_temp['DATE'].dt.year
        t_at_temp['M']=t_at_temp['DATE'].dt.month
        t_at_temp['I']=t_at_temp.index
        t_at_temp['Nazwa']=NAME
        t_at_temp['Aktywa_Trwałe_BO']=t_at_temp.apply(lambda x:0 if x['DATE']!=DATA_BO else WNA, axis=1)
        t_at_temp['Amortyzacja_BO']=0
        t_at_rozliczenie=pd.concat([t_at_rozliczenie,t_at_temp],ignore_index=True)


t_at_rozliczenie=t_at_rozliczenie.groupby(['DATE'],as_index=False)[['Aktywa_Trwałe_BO', 'Amortyzacja_BO']].sum()
t_model=pd.merge(t_model,t_at_rozliczenie,on='DATE',how='left')


In [286]:
# Rozliczenie istniejących kredytów

def CreditPeriod(credit_exp,credit_ppmt):
    if credit_ppmt!=0:
        return int((credit_exp/credit_ppmt)*12)
    else:
        return 0


CREDIT_EXP=2000000
CREDIT_PPMT=2000000
CREDIT_PRC=0.09

if CREDIT_PPMT!=0:
    okres=pd.date_range(start=DATA_BO,freq='M',periods=CreditPeriod(CREDIT_EXP,CREDIT_PPMT))
    t_credit_rozliczenie=pd.DataFrame({'DATE':okres})    
    t_credit_rozliczenie['Kredyty_BO']=t_credit_rozliczenie.apply(lambda x:0 if x['DATE']!=DATA_BO else CREDIT_EXP, axis=1)
    t_credit_rozliczenie['PPMT_BO']=-CREDIT_PPMT/12
    t_credit_rozliczenie['PRC_BO']=-(t_credit_rozliczenie['Kredyty_BO'].cumsum()+t_credit_rozliczenie['PPMT_BO'].cumsum())*CREDIT_PRC/12
else:
    t_credit_rozliczenie=t_okres[['DATE']].copy()
    t_credit_rozliczenie['Kredyty_BO']=t_credit_rozliczenie.apply(lambda x:0 if x['DATE']!=DATA_BO else CREDIT_EXP, axis=1)
    t_credit_rozliczenie['PPMT_BO']=0
    t_credit_rozliczenie['PRC_BO']=-(t_credit_rozliczenie['Kredyty_BO'].cumsum()+t_credit_rozliczenie['PPMT_BO'].cumsum())*CREDIT_PRC/12

t_model=pd.merge(t_model,t_credit_rozliczenie,on='DATE',how='left')
t_model.fillna(0,inplace=True)

In [287]:
# Rozliczenie istniejących kapitałów

KapitałPodstawowy_BO=20006569
KapitałZapasowy_BO=0
KapitałZAktualizacji_BO=0
PozostałeRezerwowe_BO=52061107
ZyskLatUbiegłych_BO=-35779951

t_model['KapitałPodstawowy_BO']=t_model.apply(lambda x:0 if x['DATE']!=DATA_BO else KapitałPodstawowy_BO,axis=1)
t_model['KapitałZapasowy_BO']=t_model.apply(lambda x:0 if x['DATE']!=DATA_BO else KapitałZapasowy_BO,axis=1)
t_model['KapitałZAktualizacji_BO']=t_model.apply(lambda x:0 if x['DATE']!=DATA_BO else KapitałZAktualizacji_BO,axis=1)
t_model['PozostałeRezerwowe_BO']=t_model.apply(lambda x:0 if x['DATE']!=DATA_BO else PozostałeRezerwowe_BO,axis=1)
t_model['ZyskLatUbiegłych_BO']=t_model.apply(lambda x:0 if x['DATE']!=DATA_BO else ZyskLatUbiegłych_BO,axis=1)

#### Planowane inwestycje

In [288]:
# Nakłady Inwestycyjne

l_ni=[
        {'Nazwa':                   'Przebudowa frontu przeładunkowego',
        'Cena [PLN]':               6000000,
        'Data OT':                  '2023-06-30',
        'Amortyzacja [okres]':      120,
        'Udział własny':            100,
        'Okres finansowania':       0,
        'Wartość rezydualna':       0},

        {'Nazwa':                   'Dodatkowy tor szeroki',
        'Cena [PLN]':               4000000,
        'Data OT':                  '2023-06-30',
        'Amortyzacja [okres]':      266,
        'Udział własny':            100,
        'Okres finansowania':       0,
        'Wartość rezydualna':       0},

        {'Nazwa':                   'Dodatkowy tor europejski',
        'Cena [PLN]':               4000000,
        'Data OT':                  '2023-06-30',
        'Amortyzacja [okres]':      266,
        'Udział własny':            100,
        'Okres finansowania':       0,
        'Wartość rezydualna':       0},

        {'Nazwa':                   'Plac utwardzony (parking)',
        'Cena [PLN]':               5000000,
        'Data OT':                  '2023-06-30',
        'Amortyzacja [okres]':      120,
        'Udział własny':            100,
        'Okres finansowania':       0,
        'Wartość rezydualna':       0},

        {'Nazwa':                   'Autocysterny',
        'Cena [PLN]':               5000000,
        'Data OT':                  '2023-06-30',
        'Amortyzacja [okres]':      60,
        'Udział własny':            100,
        'Okres finansowania':       0,
        'Wartość rezydualna':       0}
]

t_ni=pd.DataFrame(l_ni)

t_tabela=t_ni.copy()
t_tabela=t_tabela[['Nazwa', 'Cena [PLN]', 'Data OT','Udział własny','Okres finansowania','Amortyzacja [okres]']]
t_tabela.loc['RAZEM','Cena [PLN]']=t_tabela['Cena [PLN]'].sum()
t_tabela.loc['RAZEM','Nazwa']='RAZEM'
#t_tabela.fillna("",inplace=True)
t_tabela[['Cena [PLN]']]=t_tabela[['Cena [PLN]']]/1000
t_tabela_fv=t_tabela.style.format({'Cena [PLN]':'{0:,.2f}','Udział własny':'{0:,.2f}','Okres finansowania':'{0:,.2f}','Amortyzacja [okres]':'{0:,.2f}'}).hide(axis='index')
t_tabela_fv.to_latex(ASSETS / '2_Nakłady_Inwestycyjne.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}R{2cm}R{2cm}R{2cm}')
t_tabela_fv

Nazwa,Cena [PLN],Data OT,Udział własny,Okres finansowania,Amortyzacja [okres]
Przebudowa frontu przeładunkowego,"6,000.00",2023-06-30,100.00,0.00,120.00
Dodatkowy tor szeroki,"4,000.00",2023-06-30,100.00,0.00,266.00
Dodatkowy tor europejski,"4,000.00",2023-06-30,100.00,0.00,266.00
Plac utwardzony (parking),"5,000.00",2023-06-30,100.00,0.00,120.00
Autocysterny,"5,000.00",2023-06-30,100.00,0.00,60.00
RAZEM,"24,000.00",nan,nan,nan,nan


In [289]:
# Rozliczenie inwestycji

t_capex=pd.DataFrame()

for index,row in t_ni.iterrows():

    NAME=row['Nazwa']
    CENA=row['Cena [PLN]']
    DATA_OT=pd.to_datetime(row['Data OT'])
    AM=row['Amortyzacja [okres]']
    DP=row['Udział własny']
    T=row['Okres finansowania']
    RV=row['Wartość rezydualna']

    okres=pd.date_range(start=DATA_OT,freq='M',periods=AM+1)
    t_capex_temp=pd.DataFrame({'DATE':okres})
    t_capex_temp['R']=t_capex_temp['DATE'].dt.year
    t_capex_temp['M']=t_capex_temp['DATE'].dt.month
    t_capex_temp['I']=t_capex_temp.index
    t_capex_temp['Nazwa']=NAME
    t_capex_temp['Zakup']=t_capex_temp.apply(lambda x:0 if x['DATE']!=DATA_OT else CENA, axis=1)
    t_capex_temp['Amortyzacja']=t_capex_temp.apply(lambda x:0 if x['I']==0 else CENA/AM, axis=1)
    t_capex_temp['Aktywo']=t_capex_temp['Zakup'].cumsum()-t_capex_temp['Amortyzacja'].cumsum()
    t_capex_temp['Finansowanie własne']=t_capex_temp.apply(lambda x:CENA*DP/100 if x['I']==0 else 0, axis=1)
    t_capex_temp['Finansowanie obce']=t_capex_temp.apply(lambda x:CENA*(100-DP)/100 if x['I']==0 else 0, axis=1)
    t_capex_temp['PPMT']=t_capex_temp.apply(lambda x:0 if (x['I']==0 or x['I']>T) else -npf.ppmt((INDEX+MARGIN)/1200,x['I'],T,CENA*(100-DP)/100,-CENA*(RV)/100),axis=1)
    t_capex_temp['IPMT']=t_capex_temp.apply(lambda x:0 if (x['I']==0 or x['I']>T) else -npf.ipmt((INDEX+MARGIN)/1200,x['I'],T,CENA*(100-DP)/100,-CENA*(RV)/100),axis=1)
    t_capex_temp['RV']=t_capex_temp.apply(lambda x:CENA*(RV)/100 if x['DATE']==DATA_OT+pd.DateOffset(months=T) else 0, axis=1)
    t_capex_temp['WNA']=t_capex_temp.apply(lambda x:0 if x['DATE']!=DATA_OT+pd.DateOffset(months=T) else x['Aktywo'], axis=1)

    t_capex=pd.concat([t_capex,t_capex_temp],ignore_index=True)

t_capex_model=t_capex.groupby(['DATE','R'],as_index=False)[['Zakup', 'Amortyzacja','Finansowanie własne', 'Finansowanie obce', 'PPMT', 'IPMT', 'RV']].sum()

t_capex_model['Aktywo']=t_capex_model['Zakup']-t_capex_model['Amortyzacja']
t_capex_model['Pasywo']=t_capex_model['Finansowanie obce']-t_capex_model['PPMT']-t_capex_model['RV']

temp=t_capex_model[['DATE']].copy()
temp['Nakłady inwestycyjne']=-t_capex_model['Zakup']
temp['Kredyty bankowe i leasingi (inwestycje)']=t_capex_model['Finansowanie obce']
temp['Finansowanie własne']=t_capex_model['Finansowanie własne']
temp['Amortyzacja (inwestycje)']=-t_capex_model['Amortyzacja']
temp['Koszty finansowe (inwestycje)']=-t_capex_model['IPMT']
temp['Spłata kredytów i leasingów (inwestycje)']=-t_capex_model['PPMT']-t_capex_model['RV']
temp['Rzeczowe aktywa trwałe (inwestycje)']=t_capex_model['Aktywo']
temp['Zobowiązania długoterminowe (inwestycje)']=t_capex_model['Pasywo']
t_model=pd.merge(t_model,temp,how='left',on=['DATE'])
t_model.fillna(0,inplace=True)

temp=t_model[['R','Nakłady inwestycyjne','Rzeczowe aktywa trwałe (inwestycje)','Amortyzacja (inwestycje)','Kredyty bankowe i leasingi (inwestycje)','Finansowanie własne','Zobowiązania długoterminowe (inwestycje)','Spłata kredytów i leasingów (inwestycje)','Koszty finansowe (inwestycje)']].copy()
temp=temp.groupby(['R'],as_index=False)[['Nakłady inwestycyjne','Rzeczowe aktywa trwałe (inwestycje)','Amortyzacja (inwestycje)','Kredyty bankowe i leasingi (inwestycje)','Finansowanie własne','Zobowiązania długoterminowe (inwestycje)','Spłata kredytów i leasingów (inwestycje)','Koszty finansowe (inwestycje)']].sum()

temp['Rzeczowe aktywa trwałe (inwestycje)']=temp['Rzeczowe aktywa trwałe (inwestycje)'].cumsum()
temp['Zobowiązania długoterminowe (inwestycje)']=temp['Zobowiązania długoterminowe (inwestycje)'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '3_Wpływ_Inwestycji_SF.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2023,2024,2025,2026,2027,2028
Nakłady inwestycyjne,"-24,000.00",0.00,0.00,0.00,0.00,0.00
Rzeczowe aktywa trwałe (inwestycje),"22,769.55","20,308.65","17,847.74","15,386.84","12,925.94","10,965.04"
Amortyzacja (inwestycje),"-1,230.45","-2,460.90","-2,460.90","-2,460.90","-2,460.90","-1,960.90"
Kredyty bankowe i leasingi (inwestycje),0.00,0.00,0.00,0.00,0.00,0.00
Finansowanie własne,"24,000.00",0.00,0.00,0.00,0.00,0.00
Zobowiązania długoterminowe (inwestycje),0.00,0.00,0.00,0.00,0.00,0.00
Spłata kredytów i leasingów (inwestycje),0.00,0.00,0.00,0.00,0.00,0.00
Koszty finansowe (inwestycje),0.00,0.00,0.00,0.00,0.00,0.00


#### Przychody i koszty

In [290]:
# Przychody ze sprzedaży

t_pspu=t_okres[['DATE','R']].copy() # przychody ze sprzedaży usług
t_pstm=t_okres[['DATE','R']].copy() # przychody ze sprzedaży towarów i materiałow


START_Y=t_pspu['DATE'].min().year
CEL_Y=t_pspu['DATE'].max().year
TENOR_Y=CEL_Y-START_Y

PSPU_KOR_CEL=0.95
PSPU_KOR_CENA=0.95

pspu_ILE_START_Y=22000*12
pspu_ILE_CEL_Y=40000*12*PSPU_KOR_CEL
pspu_CENA=120*PSPU_KOR_CENA
pspu_CAGR=((pspu_ILE_CEL_Y/pspu_ILE_START_Y)**(1/TENOR_Y)-1)*100

PSTM_KOR_CEL=0.9
PSTM_KOR_CENA=0.9

pstm_ILE_START_Y=2000*12
pstm_ILE_CEL_Y=4000*12*PSTM_KOR_CEL
pstm_CENA=((750)*4.7+500)*PSTM_KOR_CEL
pstm_WART=(750)*4.7*PSTM_KOR_CENA
pstm_CAGR=((pstm_ILE_CEL_Y/pstm_ILE_START_Y)**(1/TENOR_Y)-1)*100


print(START_Y,CEL_Y,TENOR_Y,pspu_ILE_START_Y,pspu_ILE_CEL_Y,pspu_CENA,pspu_CAGR)
print(START_Y,CEL_Y,TENOR_Y,pstm_ILE_START_Y,pstm_ILE_CEL_Y,pstm_CENA,pstm_CAGR)


2023 2028 5 264000 456000.0 114.0 11.550670014054099
2023 2028 5 24000 43200.0 3622.5 12.474611314209483


In [291]:
# Przychody ze sprzedaży produktów i usług - kalkulacja

t_pspu['Sprzedaż usług (ilość)']=(pspu_ILE_START_Y*(1+pspu_CAGR/100)**(t_pspu['R']-START_Y))/12
t_pspu['Sprzedaż usług (przychód)']=t_pspu['Sprzedaż usług (ilość)']*pspu_CENA

# Koszty operacyjne

# Założenia
# Czynnik zmienny (ilość ton sprzedaży usług) + Czynnik stały
# 9,45	2 381 051,18	Zużycie materiałów i energii
# 4,33	2 946 749,65	Usługi obce
# 0,58	1 009 820,19	Podatki i opłaty
# 2,85	5 733 609,96	WynagrodzeniaKos
# 0,24 od wynagrodzeń	Ubezpieczenia społeczne i inne świadczenia, w tym:
# 0,22	600 829,88	Pozostałe koszty rodzajowe

KOR_ZMiE=1.5
KOR_UO=1.5
KOR_POD=1.2
KOR_WYN=2
KOR_POZ=1.2

t_pspu['ZużycieMiE']=t_pspu['Sprzedaż usług (ilość)']*9.45*KOR_ZMiE+2381051.18/12
t_pspu['UsługiO']=t_pspu['Sprzedaż usług (ilość)']*4.33*KOR_UO+2946749.65/12
t_pspu['Podatki']=t_pspu['Sprzedaż usług (ilość)']*0.58*KOR_POD+1009820.19/12
t_pspu['Wyn']=t_pspu['Sprzedaż usług (ilość)']*2.85*KOR_WYN+5733609.96/12
t_pspu['ZUS']=t_pspu['Wyn']*0.24
t_pspu['Pozostałe']=t_pspu['Sprzedaż usług (ilość)']*0.22*KOR_POZ+600829.88/12

t_model=pd.merge(t_model,t_pspu,on=['DATE','R'],how='left')

temp=t_pspu.groupby(['R'],as_index=False)[['Sprzedaż usług (ilość)','Sprzedaż usług (przychód)']].sum()


temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '4_Przychody_Usługi.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2023,2024,2025,2026,2027,2028
Sprzedaż usług (ilość),264.00,294.49,328.51,366.45,408.78,456.00
Sprzedaż usług (przychód),"30,096.00","33,572.29","37,450.11","41,775.85","46,601.24","51,984.00"


In [292]:
# Przychody ze sprzedaży towarów i materiałów - kalkulacja

t_pstm['Sprzedaż towarów (ilość)']=(pstm_ILE_START_Y*(1+pstm_CAGR/100)**(t_pstm['R']-START_Y))/12
t_pstm['Sprzedaż towarów (przychód)']=t_pstm['Sprzedaż towarów (ilość)']*pstm_CENA
t_pstm['Sprzedaż towarów (wartość)']=t_pstm['Sprzedaż towarów (ilość)']*pstm_WART
t_model=pd.merge(t_model,t_pstm,on=['DATE','R'],how='left')
temp=t_pstm.groupby(['R'],as_index=False)[['Sprzedaż towarów (ilość)','Sprzedaż towarów (przychód)','Sprzedaż towarów (wartość)']].sum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '5_Sprzedaż_TiM.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2023,2024,2025,2026,2027,2028
Sprzedaż towarów (ilość),24.00,26.99,30.36,34.15,38.41,43.20
Sprzedaż towarów (przychód),"86,940.00","97,785.43","109,983.78","123,703.83","139,135.40","156,492.00"
Sprzedaż towarów (wartość),"76,140.00","85,638.17","96,321.20","108,336.89","121,851.50","137,052.00"


In [293]:
# Mapowanie rachunku wyników

# Przychody ze sprzedaży

t_model['A. PRZYCHODY NETTO']=t_model[[
    'Sprzedaż usług (przychód)',
    'Sprzedaż towarów (przychód)']].sum(axis=1)

# Koszty operacyjne

t_model['1. Amortyzacja']=t_model[[
    'Amortyzacja_BO',
    'Amortyzacja (inwestycje)'
    ]].sum(axis=1)

t_model['2. Zużycie materiałów i energii']=-t_model[[
    'ZużycieMiE']].sum(axis=1)

t_model['3. Usługi obce']=-t_model[[
    'UsługiO']].sum(axis=1)

t_model['4. Podatki i opłaty']=-t_model[[
    'Podatki']].sum(axis=1)

t_model['5. Wynagrodzenia']=-t_model[['Wyn']].sum(axis=1)
t_model['6. Ubezpieczenia społeczne']=-t_model[['ZUS']].sum(axis=1)

t_model['7. Pozostałe koszty rodzajowe']=-t_model[['Pozostałe']].sum(axis=1)

t_model['8. Wartość sprzedanych towarów i materiałów']=-t_model[['Sprzedaż towarów (wartość)']].sum(axis=1)

t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']=t_model[[ 
    '1. Amortyzacja',
    '2. Zużycie materiałów i energii', 
    '3. Usługi obce',
    '4. Podatki i opłaty', 
    '5. Wynagrodzenia',
    '6. Ubezpieczenia społeczne', 
    '7. Pozostałe koszty rodzajowe',
    '8. Wartość sprzedanych towarów i materiałów']].sum(axis=1)

# Pozostałe przychody i koszty

t_model['C. POZOSTAŁE PRZYCHODY OPERACYJNE']=0
t_model['D. POZOSTAŁE KOSZTY OPERACYJNE']=0

# Przychody i koszty finnasowe

t_model['E. PRZYCHODY FINANSOWE']=0
t_model['F. KOSZTY FINANSOWE']=t_model[['PRC_BO','Koszty finansowe (inwestycje)']].sum(axis=1)


t_model['G. WYNIK BRUTTO']=t_model[[
    'A. PRZYCHODY NETTO',
    'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ',
    'C. POZOSTAŁE PRZYCHODY OPERACYJNE',
    'D. POZOSTAŁE KOSZTY OPERACYJNE',
    'E. PRZYCHODY FINANSOWE',
    'F. KOSZTY FINANSOWE']].sum(axis=1)

t_model['TAX']=-t_model.apply(lambda x:x['G. WYNIK BRUTTO']*0.19 if x['G. WYNIK BRUTTO']>0 else 0,axis=1)

t_model['H. PODATEK DOCHODOWY']=t_model[['TAX']].sum(axis=1)

t_model['I. WYNIK NETTO']=t_model[[
    'G. WYNIK BRUTTO',
    'H. PODATEK DOCHODOWY']].sum(axis=1)

In [294]:
# Zapotrzebowanie na kapitał obrotowy

ROTACJA_NALEŻNOŚCI=14
ROTACJA_ZAPASÓW=7
ROTACJA_ZOBOWIĄZAŃ=14

t_model['1. Należności']=(t_model['A. PRZYCHODY NETTO']*ROTACJA_NALEŻNOŚCI)/30
t_model['Nal_Shifted']=t_model['1. Należności'].shift(1,fill_value=0)
t_model['1. Należności']=t_model['1. Należności']-t_model['Nal_Shifted']

t_model['2. Zapasy']=(t_model['A. PRZYCHODY NETTO']*ROTACJA_ZAPASÓW)/30
t_model['Zap_Shifted']=t_model['2. Zapasy'].shift(1,fill_value=0)
t_model['2. Zapasy']=t_model['2. Zapasy']-t_model['Zap_Shifted']

t_model['1. Zobowiązania']=(-(t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']+t_model['1. Amortyzacja'])*ROTACJA_ZOBOWIĄZAŃ)/30
t_model['Zob_Shifted']=t_model['1. Zobowiązania'].shift(1,fill_value=0)
t_model['1. Zobowiązania']=t_model['1. Zobowiązania']-t_model['Zob_Shifted']

t_model['A. AKTYWA BIEŻĄCE']=t_model['1. Należności']+t_model['2. Zapasy']
t_model['B. PASYWA BIEŻĄCE']=t_model['1. Zobowiązania']
t_model['C. KAPITAŁ OBROTOWY NETTO']=t_model['A. AKTYWA BIEŻĄCE']-t_model['B. PASYWA BIEŻĄCE']

In [295]:
# Przepływy pieniężne


t_model['1. Kapitał zakładowy']=t_model[['KapitałPodstawowy_BO','KapitałZapasowy_BO','KapitałZAktualizacji_BO','PozostałeRezerwowe_BO', 'ZyskLatUbiegłych_BO']].sum(axis=1)
t_model['2. Kredyty bankowe']=t_model[['Kredyty bankowe i leasingi (inwestycje)','Kredyty_BO']].sum(axis=1)
t_model['3. Leasing finansowy']=0
t_model['4. Inne']=0
t_model['5. Przychody ze sprzedaży']=t_model['A. PRZYCHODY NETTO']+t_model['C. POZOSTAŁE PRZYCHODY OPERACYJNE']+t_model['E. PRZYCHODY FINANSOWE']

t_model['A. WPŁYWY PIENIĘŻNE OGÓŁEM']=t_model[['1. Kapitał zakładowy', '2. Kredyty bankowe', '3. Leasing finansowy','4. Inne', '5. Przychody ze sprzedaży']].sum(axis=1)



t_model['1. Nakłady inwestycyjne']=t_model['Nakłady inwestycyjne']-t_model['Aktywa_Trwałe_BO']
t_model['2. Koszty operacyjne (bez amortyzacji)']=t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']-t_model['1. Amortyzacja']
t_model['3. Pozostałe koszty operacyjne']=t_model['D. POZOSTAŁE KOSZTY OPERACYJNE']
t_model['4. Spłaty kredytów i leasingów']=t_model[['Spłata kredytów i leasingów (inwestycje)','PPMT_BO']].sum(axis=1)
t_model['5. Odsetki']=t_model['F. KOSZTY FINANSOWE']
t_model['6. Podatek dochodowy']=t_model['H. PODATEK DOCHODOWY']

t_model['B. WYDATKI PIENIĘŻNE OGÓŁEM']=t_model[['1. Nakłady inwestycyjne', '2. Koszty operacyjne (bez amortyzacji)','3. Pozostałe koszty operacyjne', '4. Spłaty kredytów i leasingów','5. Odsetki', '6. Podatek dochodowy']].sum(axis=1)

t_model['C. ZAPOTRZEBOWANIE NA KON']=-t_model['C. KAPITAŁ OBROTOWY NETTO']

t_model['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']=t_model['A. WPŁYWY PIENIĘŻNE OGÓŁEM']+t_model['B. WYDATKI PIENIĘŻNE OGÓŁEM']+t_model['C. ZAPOTRZEBOWANIE NA KON']
t_model['E. SKUMULOWANE SALDO ŚP']=t_model['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']

In [296]:
# Mapowanie bilansu - Aktywa

t_model['1. Wartości niematerialne i prawne']=0
t_model['2. Rzeczowe aktywa trwałe']=t_model[['Rzeczowe aktywa trwałe (inwestycje)','Aktywa_Trwałe_BO', 'Amortyzacja_BO']].sum(axis=1)
t_model['3. Należności długoterminowe']=0.0
t_model['4. Długoterminowe rozliczenia międzyokresowe']=0

t_model['A. AKTYWA TRWAŁE']=t_model[[ 
    '1. Wartości niematerialne i prawne',
    '2. Rzeczowe aktywa trwałe',
    '3. Należności długoterminowe',
    '4. Długoterminowe rozliczenia międzyokresowe']].sum(axis=1)

t_model['1. Zapasy']=t_model[['2. Zapasy']].sum(axis=1)
t_model['2. Należności krótkoterminowe']=t_model[['1. Należności']].sum(axis=1)
t_model['3. Inwestycje krótkoterminowe']=t_model['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']
t_model['4. Krótkoterminowe rozliczenia międzyokresowe']=0.0

t_model['B. AKTYWA OBROTOWE']=t_model[[ 
    '1. Zapasy',
    '2. Należności krótkoterminowe',
    '3. Inwestycje krótkoterminowe',
    '4. Krótkoterminowe rozliczenia międzyokresowe']].sum(axis=1)

t_model['AKTYWA RAZEM']=t_model[['A. AKTYWA TRWAŁE','B. AKTYWA OBROTOWE']].sum(axis=1)

In [297]:
# Mapowanie bilansu - Pasywa

t_model['1. Kapitał podstawowy']=t_model['KapitałPodstawowy_BO']
t_model['2. Kapitał zapasowy']=t_model['KapitałZapasowy_BO']
t_model['3. Kapitał z aktualizacji wyceny']=t_model['KapitałZAktualizacji_BO']
t_model['4. Pozostałe kapitały rezerwowe']=t_model['PozostałeRezerwowe_BO']
t_model['5. Zysk (strata) z lat ubiegłych']=t_model['ZyskLatUbiegłych_BO']
t_model['6. Zysk (strata)']=t_model['I. WYNIK NETTO']
t_model['7. Odpisy w ciągu roku']=0

t_model['A. KAPITAŁ WŁASNY']=t_model[[ 
    '1. Kapitał podstawowy',
    '2. Kapitał zapasowy',
    '3. Kapitał z aktualizacji wyceny',
    '4. Pozostałe kapitały rezerwowe',
    '5. Zysk (strata) z lat ubiegłych',
    '6. Zysk (strata)',
    '7. Odpisy w ciągu roku']].sum(axis=1)

t_model['1. Rezerwy na zobowiązania']=0
t_model['2. Zobowiązania długoterminowe']=t_model[['Zobowiązania długoterminowe (inwestycje)','Kredyty_BO', 'PPMT_BO']].sum(axis=1)
t_model['3. Zobowiązania krótkoterminowe']=t_model['1. Zobowiązania']
t_model['4. Rozliczenia międzyokresowe']=0
t_model['5. Fundusze specjalne']=0

t_model['B. ZOBOWIĄZANIA I REZERWY']=t_model[[ 
    '1. Rezerwy na zobowiązania',
    '2. Zobowiązania długoterminowe',
    '3. Zobowiązania krótkoterminowe',
    '4. Rozliczenia międzyokresowe',
    '5. Fundusze specjalne']].sum(axis=1)

t_model['PASYWA RAZEM']=t_model[['A. KAPITAŁ WŁASNY', 'B. ZOBOWIĄZANIA I REZERWY']].sum(axis=1)

In [298]:
# Raport roczny
temp=t_model[[
       'R',
       'A. PRZYCHODY NETTO',
       'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ',
       '1. Amortyzacja',
       '2. Zużycie materiałów i energii',
       '3. Usługi obce',
       '4. Podatki i opłaty',
       '5. Wynagrodzenia',
       '6. Ubezpieczenia społeczne',
       '7. Pozostałe koszty rodzajowe',
       '8. Wartość sprzedanych towarów i materiałów',
       'C. POZOSTAŁE PRZYCHODY OPERACYJNE',
       'D. POZOSTAŁE KOSZTY OPERACYJNE',
       'E. PRZYCHODY FINANSOWE',
       'F. KOSZTY FINANSOWE',
       'G. WYNIK BRUTTO',
       'H. PODATEK DOCHODOWY',
       'I. WYNIK NETTO']].copy()

temp=temp.groupby(['R'],as_index=False)[[
       'A. PRZYCHODY NETTO',
       'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ',
       '1. Amortyzacja',
       '2. Zużycie materiałów i energii',
       '3. Usługi obce',
       '4. Podatki i opłaty',
       '5. Wynagrodzenia',
       '6. Ubezpieczenia społeczne',
       '7. Pozostałe koszty rodzajowe',
       '8. Wartość sprzedanych towarów i materiałów',
       'C. POZOSTAŁE PRZYCHODY OPERACYJNE',
       'D. POZOSTAŁE KOSZTY OPERACYJNE',
       'E. PRZYCHODY FINANSOWE',
       'F. KOSZTY FINANSOWE',
       'G. WYNIK BRUTTO',
       'H. PODATEK DOCHODOWY',
       'I. WYNIK NETTO']].sum()


temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '6_PL.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2023,2024,2025,2026,2027,2028
A. PRZYCHODY NETTO,"117,036.00","131,357.72","147,433.89","165,479.68","185,736.64","208,476.00"
B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ,"-103,732.85","-112,836.58","-123,995.80","-137,100.44","-151,829.78","-167,821.32"
1. Amortyzacja,"-5,968.45","-4,698.90","-4,198.90","-4,198.90","-4,198.90","-3,634.90"
2. Zużycie materiałów i energii,"-6,123.25","-6,555.50","-7,037.68","-7,575.55","-8,175.55","-8,844.85"
3. Usługi obce,"-4,661.43","-4,859.49","-5,080.42","-5,326.87","-5,601.79","-5,908.47"
4. Podatki i opłaty,"-1,193.56","-1,214.79","-1,238.46","-1,264.87","-1,294.33","-1,327.20"
5. Wynagrodzenia,"-7,238.41","-7,412.22","-7,606.12","-7,822.40","-8,063.67","-8,332.81"
6. Ubezpieczenia społeczne,"-1,737.22","-1,778.93","-1,825.47","-1,877.38","-1,935.28","-1,999.87"
7. Pozostałe koszty rodzajowe,-670.53,-678.58,-687.56,-697.57,-708.75,-721.21
8. Wartość sprzedanych towarów i materiałów,"-76,140.00","-85,638.17","-96,321.20","-108,336.89","-121,851.50","-137,052.00"


In [299]:
# Zapotrzebowanie ma KO Tabela

temp=t_model[['R','A. AKTYWA BIEŻĄCE', '1. Należności', '2. Zapasy', 'B. PASYWA BIEŻĄCE', '1. Zobowiązania']].copy()
temp=temp.groupby(['R'],as_index=False)[['A. AKTYWA BIEŻĄCE', '1. Należności', '2. Zapasy', 'B. PASYWA BIEŻĄCE', '1. Zobowiązania']].sum()
temp=temp.cumsum()
temp['C. KAPITAŁ OBROTOWY NETTO']=temp['A. AKTYWA BIEŻĄCE']-temp['B. PASYWA BIEŻĄCE']
temp['temp']=temp.shift(1)['C. KAPITAŁ OBROTOWY NETTO']
temp['D. ZAPOTRZEBOWANIE NA KON']=-temp.apply(lambda x:x['C. KAPITAŁ OBROTOWY NETTO'] if x['R']==DATE_START.year else x['C. KAPITAŁ OBROTOWY NETTO']-x['temp'],axis=1)
temp.drop(['temp'],axis=1,inplace=True)

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '7_KON.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2023,2024,2025,2026,2027,2028
A. AKTYWA BIEŻĄCE,"6,827.10","7,662.53","8,600.31","9,652.98","10,834.64","12,161.10"
1. Należności,"4,551.40","5,108.36","5,733.54","6,435.32","7,223.09","8,107.40"
2. Zapasy,"2,275.70","2,554.18","2,866.77","3,217.66","3,611.55","4,053.70"
B. PASYWA BIEŻĄCE,"4,361.86","4,531.94","4,985.35","5,494.97","6,067.78","6,628.85"
1. Zobowiązania,"4,361.86","4,531.94","4,985.35","5,494.97","6,067.78","6,628.85"
C. KAPITAŁ OBROTOWY NETTO,"2,465.24","3,130.60","3,614.96","4,158.01","4,766.86","5,532.25"
D. ZAPOTRZEBOWANIE NA KON,"-2,465.24",-665.36,-484.36,-543.05,-608.85,-765.39


In [300]:
# Cash Flow - Tabela


temp=t_model[['R','A. WPŁYWY PIENIĘŻNE OGÓŁEM', '1. Kapitał zakładowy',
       '2. Kredyty bankowe', '3. Leasing finansowy', '4. Inne',
       '5. Przychody ze sprzedaży', 'B. WYDATKI PIENIĘŻNE OGÓŁEM',
       '1. Nakłady inwestycyjne', '2. Koszty operacyjne (bez amortyzacji)',
       '3. Pozostałe koszty operacyjne', '4. Spłaty kredytów i leasingów',
       '5. Odsetki', '6. Podatek dochodowy', 'C. ZAPOTRZEBOWANIE NA KON',
       'D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']]

temp=temp.groupby(['R'],as_index=False)[['A. WPŁYWY PIENIĘŻNE OGÓŁEM', '1. Kapitał zakładowy',
       '2. Kredyty bankowe', '3. Leasing finansowy', '4. Inne',
       '5. Przychody ze sprzedaży', 'B. WYDATKI PIENIĘŻNE OGÓŁEM',
       '1. Nakłady inwestycyjne', '2. Koszty operacyjne (bez amortyzacji)',
       '3. Pozostałe koszty operacyjne', '4. Spłaty kredytów i leasingów',
       '5. Odsetki', '6. Podatek dochodowy', 'C. ZAPOTRZEBOWANIE NA KON',
       'D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']].sum()

temp['E. SKUMULOWANE SALDO ŚP']=temp['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH'].cumsum()
temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '8_CF.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2023,2024,2025,2026,2027,2028
A. WPŁYWY PIENIĘŻNE OGÓŁEM,"155,323.73","131,357.72","147,433.89","165,479.68","185,736.64","208,476.00"
1. Kapitał zakładowy,"36,287.72",0.00,0.00,0.00,0.00,0.00
2. Kredyty bankowe,"2,000.00",0.00,0.00,0.00,0.00,0.00
3. Leasing finansowy,0.00,0.00,0.00,0.00,0.00,0.00
4. Inne,0.00,0.00,0.00,0.00,0.00,0.00
5. Przychody ze sprzedaży,"117,036.00","131,357.72","147,433.89","165,479.68","185,736.64","208,476.00"
B. WYDATKI PIENIĘŻNE OGÓŁEM,"-155,228.82","-111,656.69","-124,250.14","-138,293.60","-154,073.18","-171,910.80"
1. Nakłady inwestycyjne,"-52,870.00",0.00,0.00,0.00,0.00,0.00
2. Koszty operacyjne (bez amortyzacji),"-97,764.40","-108,137.68","-119,796.90","-132,901.54","-147,630.88","-164,186.42"
3. Pozostałe koszty operacyjne,0.00,0.00,0.00,0.00,0.00,0.00


In [301]:
# Raport roczny
temp=t_model[[
       'R',
       'A. AKTYWA TRWAŁE',
       '1. Wartości niematerialne i prawne',
       '2. Rzeczowe aktywa trwałe',
       '3. Należności długoterminowe',
       '4. Długoterminowe rozliczenia międzyokresowe',
       'B. AKTYWA OBROTOWE',
       '1. Zapasy',
       '2. Należności krótkoterminowe',
       '3. Inwestycje krótkoterminowe',
       '4. Krótkoterminowe rozliczenia międzyokresowe',
       'AKTYWA RAZEM']].copy()

temp=temp.groupby(['R'],as_index=False)[[
       'A. AKTYWA TRWAŁE',
       '1. Wartości niematerialne i prawne',
       '2. Rzeczowe aktywa trwałe',
       '3. Należności długoterminowe',
       '4. Długoterminowe rozliczenia międzyokresowe',
       'B. AKTYWA OBROTOWE',
       '1. Zapasy',
       '2. Należności krótkoterminowe',
       '3. Inwestycje krótkoterminowe',
       '4. Krótkoterminowe rozliczenia międzyokresowe',
       'AKTYWA RAZEM']].sum()

temp=temp.cumsum()
temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '9_Aktywa.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2023,2024,2025,2026,2027,2028
A. AKTYWA TRWAŁE,"46,901.55","42,202.65","38,003.74","33,804.84","29,605.94","25,971.04"
1. Wartości niematerialne i prawne,0.00,0.00,0.00,0.00,0.00,0.00
2. Rzeczowe aktywa trwałe,"46,901.55","42,202.65","38,003.74","33,804.84","29,605.94","25,971.04"
3. Należności długoterminowe,0.00,0.00,0.00,0.00,0.00,0.00
4. Długoterminowe rozliczenia międzyokresowe,0.00,0.00,0.00,0.00,0.00,0.00
B. AKTYWA OBROTOWE,"4,456.77","24,327.86","47,965.03","75,660.74","107,897.01","145,023.28"
1. Zapasy,"2,275.70","2,554.18","2,866.77","3,217.66","3,611.55","4,053.70"
2. Należności krótkoterminowe,"4,551.40","5,108.36","5,733.54","6,435.32","7,223.09","8,107.40"
3. Inwestycje krótkoterminowe,"-2,370.33","16,665.33","39,364.72","66,007.76","97,062.37","132,862.18"
4. Krótkoterminowe rozliczenia międzyokresowe,0.00,0.00,0.00,0.00,0.00,0.00


In [302]:
# Raport roczny
temp=t_model[[
       'R',
      'A. KAPITAŁ WŁASNY',
       '1. Kapitał podstawowy',
       '2. Kapitał zapasowy',
       '3. Kapitał z aktualizacji wyceny',
       '4. Pozostałe kapitały rezerwowe',
       '5. Zysk (strata) z lat ubiegłych',
       '6. Zysk (strata)',
       '7. Odpisy w ciągu roku',
       'B. ZOBOWIĄZANIA I REZERWY',
       '1. Rezerwy na zobowiązania',
       '2. Zobowiązania długoterminowe',
       '3. Zobowiązania krótkoterminowe',
       '4. Rozliczenia międzyokresowe',
       '5. Fundusze specjalne',
       'PASYWA RAZEM']].copy()

temp=temp.groupby(['R'],as_index=False)[[
       'A. KAPITAŁ WŁASNY',
       '1. Kapitał podstawowy',
       '2. Kapitał zapasowy',
       '3. Kapitał z aktualizacji wyceny',
       '4. Pozostałe kapitały rezerwowe',
       '5. Zysk (strata) z lat ubiegłych',
       '6. Zysk (strata)',
       '7. Odpisy w ciągu roku',
       'B. ZOBOWIĄZANIA I REZERWY',
       '1. Rezerwy na zobowiązania',
       '2. Zobowiązania długoterminowe',
       '3. Zobowiązania krótkoterminowe',
       '4. Rozliczenia międzyokresowe',
       '5. Fundusze specjalne',
       'PASYWA RAZEM']].sum()

temp=temp.cumsum()
temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '10_Pasywa.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2023,2024,2025,2026,2027,2028
A. KAPITAŁ WŁASNY,"46,996.45","61,998.57","80,983.43","103,970.61","131,435.17","164,365.46"
1. Kapitał podstawowy,"20,006.57","20,006.57","20,006.57","20,006.57","20,006.57","20,006.57"
2. Kapitał zapasowy,0.00,0.00,0.00,0.00,0.00,0.00
3. Kapitał z aktualizacji wyceny,0.00,0.00,0.00,0.00,0.00,0.00
4. Pozostałe kapitały rezerwowe,"52,061.11","52,061.11","52,061.11","52,061.11","52,061.11","52,061.11"
5. Zysk (strata) z lat ubiegłych,"-35,779.95","-35,779.95","-35,779.95","-35,779.95","-35,779.95","-35,779.95"
6. Zysk (strata),"10,708.73","25,710.85","44,695.70","67,682.88","95,147.44","128,077.74"
7. Odpisy w ciągu roku,0.00,0.00,0.00,0.00,0.00,0.00
B. ZOBOWIĄZANIA I REZERWY,"4,361.86","4,531.94","4,985.35","5,494.97","6,067.78","6,628.85"
1. Rezerwy na zobowiązania,0.00,0.00,0.00,0.00,0.00,0.00


In [303]:
def rating(ICR):
       if ICR>12.5:
              rt='AAA'
              spread=0.63
              return[rt,spread]
       elif ICR>9.5 and ICR<=12.5:
              rt='AA'
              spread=0.78
              return[rt,spread]
       elif ICR>7.5 and ICR<=9.5:
              rt='A+'
              spread=0.98
              return[rt,spread]
       elif ICR>6 and ICR<=7.5:
              rt='A'
              spread=1.08
              return[rt,spread]
       elif ICR>4.5 and ICR<=6:
              rt='A-'
              spread=1.22
              return[rt,spread]
       elif ICR>4 and ICR<=4.5:
              rt='BBB'
              spread=1.56
              return[rt,spread]
       elif ICR>3.5 and ICR<=4:
              rt='BB+'
              spread=2
              return[rt,spread]
       elif ICR>3 and ICR<=3.5:
              rt='BB'
              spread=2.4
              return[rt,spread]
       elif ICR>2.5 and ICR<=3:
              rt='B+'
              spread=3.51
              return[rt,spread]
       elif ICR>2 and ICR<=2.5:
              rt='B'
              spread=4.21
              return[rt,spread]
       elif ICR>1.5 and ICR<=2:
              rt='B-'
              spread=5.15
              return[rt,spread]
       elif ICR>1.25 and ICR<=1.5:
              rt='CCC'
              spread=8.2
              return[rt,spread]
       elif ICR>0.8 and ICR<=1.25:
              rt='CC'
              spread=8.64
              return[rt,spread]
       elif ICR>0.5 and ICR<=0.8:
              rt='C'
              spread=11.34
              return[rt,spread]
       else:
              rt='D'
              spread=15.12
              return[rt,spread]


In [304]:
# Wycena DCF

PRC_OBLIGACJE_10YPLN=6.42
ERP=5.08 # Rating A2 Moodys
BETA=0.48 # Unleveraged for Gas&Oil distribution
WIBOR_3M=7.28
TAX_RATE=0.19


t_wacc=t_model[[
       'R',
       '1. Amortyzacja',
       'F. KOSZTY FINANSOWE',
       'H. PODATEK DOCHODOWY',
       'I. WYNIK NETTO',
       'A. KAPITAŁ WŁASNY',
       'B. ZOBOWIĄZANIA I REZERWY',
       'C. ZAPOTRZEBOWANIE NA KON',
       'Nakłady inwestycyjne' ]].copy()


t_wacc=t_wacc.groupby(['R'],as_index=False)[[
       '1. Amortyzacja',
       'F. KOSZTY FINANSOWE',
       'H. PODATEK DOCHODOWY',
       'I. WYNIK NETTO',
       'A. KAPITAŁ WŁASNY',
       'B. ZOBOWIĄZANIA I REZERWY',
       'C. ZAPOTRZEBOWANIE NA KON',
       'Nakłady inwestycyjne']].sum()

t_wacc['A. KAPITAŁ WŁASNY']=t_wacc['A. KAPITAŁ WŁASNY'].cumsum()
t_wacc['B. ZOBOWIĄZANIA I REZERWY']=t_wacc['B. ZOBOWIĄZANIA I REZERWY'].cumsum()

t_wacc['KAPITAŁ WŁASNY (E)']=t_wacc['A. KAPITAŁ WŁASNY']
t_wacc['KAPITAŁ OBCY (D)']=t_wacc['B. ZOBOWIĄZANIA I REZERWY']
t_wacc['EBIT']=t_wacc['I. WYNIK NETTO']-t_wacc['F. KOSZTY FINANSOWE']-t_wacc['H. PODATEK DOCHODOWY']
t_wacc['ICR (Interest Coverage Ratio)']=t_wacc.apply(lambda x:100 if x['F. KOSZTY FINANSOWE']==0 else -x['EBIT']/x['F. KOSZTY FINANSOWE'],axis=1)
t_wacc['Wskaźnik D/E']=t_wacc['KAPITAŁ OBCY (D)']/t_wacc['KAPITAŁ WŁASNY (E)']
t_wacc['Wskaźnik D/(D+E)']=t_wacc['KAPITAŁ OBCY (D)']/(t_wacc['KAPITAŁ WŁASNY (E)']+t_wacc['KAPITAŁ OBCY (D)'])
t_wacc['Stopa podatkowa']=TAX_RATE
t_wacc['Stopa wolna od ryzyka (Obligacje)']=PRC_OBLIGACJE_10YPLN
t_wacc['ERP (Equity Risk Premium)']=ERP
t_wacc['Wskaźnik Beta (nielewarowana)']=BETA
t_wacc['Wskaźnik Beta (lewarowana)']=BETA*(1+(t_wacc['Wskaźnik D/E']*(1-TAX_RATE)))
t_wacc['Rating kredytowy']=t_wacc.apply(lambda x:rating(x['ICR (Interest Coverage Ratio)'])[0],axis=1)
t_wacc['Marża ryzyka']=t_wacc.apply(lambda x:rating(x['ICR (Interest Coverage Ratio)'])[1],axis=1)
t_wacc['Stopa bazowa (WIBOR)']=WIBOR_3M
t_wacc['KOSZT KAPITAŁU WŁASNEGO']=PRC_OBLIGACJE_10YPLN+t_wacc['ERP (Equity Risk Premium)']*t_wacc['Wskaźnik Beta (lewarowana)']

t_wacc['KOSZT KAPITAŁU OBCEGO']=t_wacc['Stopa bazowa (WIBOR)']+t_wacc['Marża ryzyka']
t_wacc['WACC']=t_wacc['KOSZT KAPITAŁU WŁASNEGO']*(1-t_wacc['Wskaźnik D/(D+E)'])+t_wacc['KOSZT KAPITAŁU OBCEGO']*(t_wacc['Wskaźnik D/(D+E)'])
t_wacc['Korekta ryzyka']=1.5
t_wacc['CapRate(WACC)']=t_wacc['WACC']+t_wacc['Korekta ryzyka']

t_wacc['Wydatki inwestycyjne (CapEx)']=-t_wacc['Nakłady inwestycyjne']
t_wacc['Amortyzacja']=-t_wacc['1. Amortyzacja']
t_wacc['Zmiana KO (ChWC)']=-t_wacc['C. ZAPOTRZEBOWANIE NA KON']

t_wacc['FCFF (Wolne przepływy pieniężne)']=t_wacc['EBIT']*(1-TAX_RATE)+t_wacc['Amortyzacja']-t_wacc['Wydatki inwestycyjne (CapEx)']-t_wacc['Zmiana KO (ChWC)']

last_cf=t_wacc['FCFF (Wolne przepływy pieniężne)'].iat[-1]
last_cap=t_wacc['CapRate(WACC)'].iat[-1]
ResValue=-npf.pv(last_cap/100,15,last_cf)

t_wacc['Wartość rezydualna (RV)']=t_wacc.apply(lambda x:0 if x['R']<DATE_R+5 else ResValue,axis=1)

t_wacc['DCF']=(t_wacc['FCFF (Wolne przepływy pieniężne)']+t_wacc['Wartość rezydualna (RV)'])/(1+t_wacc['CapRate(WACC)']/100)**(t_wacc['R']-DATE_R)


In [305]:
# ----------- Tabela -------------------------------------------------------------

temp=t_wacc[['R','KAPITAŁ WŁASNY (E)', 'KAPITAŁ OBCY (D)', 'EBIT',
       'ICR (Interest Coverage Ratio)', 'Wskaźnik D/E', 'Wskaźnik D/(D+E)',
       'Stopa podatkowa', 'Stopa wolna od ryzyka (Obligacje)',
       'ERP (Equity Risk Premium)', 'Wskaźnik Beta (nielewarowana)',
       'Wskaźnik Beta (lewarowana)', 'Marża ryzyka',
       'Stopa bazowa (WIBOR)', 'KOSZT KAPITAŁU WŁASNEGO',
       'KOSZT KAPITAŁU OBCEGO', 'WACC', 'Korekta ryzyka', 'CapRate(WACC)']].copy()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp[['KAPITAŁ WŁASNY (E)', 'KAPITAŁ OBCY (D)', 'EBIT']]=temp[['KAPITAŁ WŁASNY (E)', 'KAPITAŁ OBCY (D)', 'EBIT']]/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()

tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '11_WACC.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2023,2024,2025,2026,2027,2028
KAPITAŁ WŁASNY (E),"46,996.45","61,998.57","80,983.43","103,970.61","131,435.17","164,365.46"
KAPITAŁ OBCY (D),"4,361.86","4,531.94","4,985.35","5,494.97","6,067.78","6,628.85"
EBIT,"13,303.15","18,521.14","23,438.09","28,379.24","33,906.87","40,654.68"
ICR (Interest Coverage Ratio),161.25,100.00,100.00,100.00,100.00,100.00
Wskaźnik D/E,0.09,0.07,0.06,0.05,0.05,0.04
Wskaźnik D/(D+E),0.08,0.07,0.06,0.05,0.04,0.04
Stopa podatkowa,0.19,0.19,0.19,0.19,0.19,0.19
Stopa wolna od ryzyka (Obligacje),6.42,6.42,6.42,6.42,6.42,6.42
ERP (Equity Risk Premium),5.08,5.08,5.08,5.08,5.08,5.08
Wskaźnik Beta (nielewarowana),0.48,0.48,0.48,0.48,0.48,0.48


In [306]:
# ----------- Tabela -------------------------------------------------------------

temp=t_wacc[['R','EBIT','Stopa podatkowa','Wydatki inwestycyjne (CapEx)', 'Amortyzacja', 'Zmiana KO (ChWC)',
       'FCFF (Wolne przepływy pieniężne)', 'Wartość rezydualna (RV)', 'DCF']].copy()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp[['EBIT','Wydatki inwestycyjne (CapEx)', 'Amortyzacja', 'Zmiana KO (ChWC)','FCFF (Wolne przepływy pieniężne)', 'Wartość rezydualna (RV)', 'DCF']]=temp[['EBIT','Wydatki inwestycyjne (CapEx)', 'Amortyzacja', 'Zmiana KO (ChWC)','FCFF (Wolne przepływy pieniężne)', 'Wartość rezydualna (RV)', 'DCF']]/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()

tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '12_DCF.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2023,2024,2025,2026,2027,2028
EBIT,"13,303.15","18,521.14","23,438.09","28,379.24","33,906.87","40,654.68"
Stopa podatkowa,0.19,0.19,0.19,0.19,0.19,0.19
Wydatki inwestycyjne (CapEx),"24,000.00",-0.00,-0.00,-0.00,-0.00,-0.00
Amortyzacja,"5,968.45","4,698.90","4,198.90","4,198.90","4,198.90","3,634.90"
Zmiana KO (ChWC),"2,465.24",665.36,484.36,543.05,608.85,765.39
FCFF (Wolne przepływy pieniężne),"-9,721.23","19,035.66","22,699.39","26,643.04","31,054.61","35,799.80"
Wartość rezydualna (RV),0.00,0.00,0.00,0.00,0.00,"266,216.93"
DCF,"-9,721.23","17,238.02","18,618.08","19,795.17","20,902.18","184,170.90"


In [307]:
t_wacc['DCF'].sum()

251003119.98241478